In [ ]:
import sys
sys.path.append('..')

In [1]:
running = False
try:
    sock.close()
except:
    pass

In [3]:
from new_parser import construct, match, match_exact

In [4]:
from utils import SocketReader

In [5]:
import math

In [6]:
rmeote_offer = """v=0
o=- 4912609901749366419 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 60976 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 35.226.7.11
a=candidate:1864292712 1 udp 2113937151 63118ac4-e909-4fa4-84e1-2bcf0c4999e3.local 60976 typ host generation 0 network-cost 999
a=candidate:916458619 1 udp 1677729535 35.226.7.11 60976 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:pXw2
a=ice-pwd:A21AA3M8FWVJQ/QFDFxon+tg
a=ice-options:trickle
a=fingerprint:sha-256 FB:53:9E:49:62:01:25:B8:4F:3E:2B:F7:8C:C0:3D:FC:52:35:6E:9A:6A:4D:3B:26:F8:71:1C:FC:6E:4D:F0:F4
a=setup:actpass
a=mid:0
a=sctp-port:5000
a=max-message-size:262144

"""

In [7]:
for line in rmeote_offer.splitlines():
    if 'a=candidate:' in line and '.local' not in line:
        res = line.split(' ')
        target_ip = res[4]
        target_port = int(res[5])
    if 'a=ice-ufrag:' in line:
        remote_ufrag = line.split(':')[1]
    if 'a=ice-pwd:' in line:
        remote_pwd = line.split(':')[1]

target_ip_num = 0
for i, seg in enumerate(target_ip.split('.')):
    target_ip_num += int(seg) * (256**(3-i))

target_ip, target_port, remote_ufrag, remote_pwd

('35.226.7.11', 60976, 'pXw2', 'A21AA3M8FWVJQ/QFDFxon+tg')

In [8]:
import random, string
len_remote_ufrag = len(remote_ufrag)
# len_remote_ufrag = 4 # chrome
local_ufrag = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16 - len_remote_ufrag - 1))
local_pwd = 'AgLw44VDUK531eYVCMpZMAjF'
local_ufrag

'bpSftNCsadQ'

In [9]:
from stun import do_stun
import random
sport = random.randrange(30000, 50000)
ip, port = do_stun(sport)
f"found {ip}:{port} while source port is {sport}"

'found 87.212.158.114:45459 while source port is 45459'

In [10]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
import OpenSSL.crypto

own_cert_private_key = ec.generate_private_key(ec.SECP256R1())

own_cert_private_key_bytes = own_cert_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)


privkey = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, own_cert_private_key_bytes)

pubkey = OpenSSL.crypto.load_publickey(OpenSSL.crypto.FILETYPE_PEM,
                                     own_cert_private_key.public_key().public_bytes(
                                         encoding=serialization.Encoding.PEM,
                                         format=serialization.PublicFormat.SubjectPublicKeyInfo,
                                     )
                                    )

local_cert = OpenSSL.crypto.X509()
local_cert.set_version(2)
local_cert.gmtime_adj_notBefore(0)
local_cert.gmtime_adj_notAfter(1 * 24 * 60 * 60)
local_cert.get_issuer().CN = 'WebRTC'
local_cert.get_subject().CN = 'WebRTC'
local_cert.set_serial_number(random.randrange(1<<63, 1<<64))
local_cert.set_pubkey(pubkey)
local_cert.sign(privkey, 'sha256') 
print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, local_cert).decode('utf-8')) 
local_cert_sha = local_cert.digest('sha256').decode('utf-8')
local_cert_sha

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            b8:7b:0d:9a:c9:14:2e:f9
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jul  8 12:54:15 2024 GMT
            Not After : Jul  9 12:54:15 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:da:5d:9c:c0:ee:1f:5f:9b:59:dd:93:96:ba:5a:
                    54:a5:0d:45:6e:1f:15:6e:e1:c7:d3:0b:90:3e:3b:
                    b4:1c:d9:ec:6c:9f:8a:94:55:80:5d:7f:57:63:ac:
                    83:cb:fd:56:c3:66:df:97:ac:d3:12:d9:81:91:5e:
                    25:87:88:73:8d
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:46:02:21:00:d4:36:94:84:01:88:fd:2c:04:06:d1:78:12:
        6e:36:82:4d:4c:c3:c9:22:c2:9c:4f

'FD:04:01:01:24:3E:40:8E:F7:4A:A2:A6:4C:78:5D:FE:D3:E5:1E:30:97:1D:02:01:F0:BD:1A:59:F6:28:39:D3'

In [11]:
local_offer = f"""v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application {port} UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 {ip}
a=candidate:3055715734 1 udp 1677729535 {ip} {port} typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:{local_ufrag}
a=ice-pwd:{local_pwd}
a=ice-options:trickle
a=fingerprint:sha-256 {local_cert_sha}
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144
"""
print(local_offer)

v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 45459 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 87.212.158.114
a=candidate:3055715734 1 udp 1677729535 87.212.158.114 45459 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:bpSftNCsadQ
a=ice-pwd:AgLw44VDUK531eYVCMpZMAjF
a=ice-options:trickle
a=fingerprint:sha-256 FD:04:01:01:24:3E:40:8E:F7:4A:A2:A6:4C:78:5D:FE:D3:E5:1E:30:97:1D:02:01:F0:BD:1A:59:F6:28:39:D3
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144



In [12]:
stun_header_spec = """
2@zeroes
-14@message_type
-16@message_length
-32@magic_cookie
-96@transaction_id
"""

padding_0_attribute_spec = """
...(
    16==0x20@_selector-16@xor_mapped_ip(
        8@zeroes
        -8@family
        -16@port
        -32@ip
    )
    |16==0x06@_selector-16@username()
    |16==0x08@_selector-16@message_integrity(20B@hmac)
    |16==0x25@_selector-16@use_candidate()
    |16==0x8029@_selector-16@ice_controlled(8B@tie_breaker)
    |16==0x802a@_selector-16@ice_controlling(8B@tie_breaker)
    |16==0x24@_selector-16@priority(4B@priority)
    |16==0x8028@_selector-16@fingerprint(4B@crc32)
    |16@selector-16@data()
)
"""

import random
magic_cookie = 0x2112A442 # constant

In [13]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hmac import HMAC

def get_message_integrity(msg, key):
    msg = bytearray(msg)
    msg[3] = len(msg) + 24 - 20
    hm = HMAC(key, hashes.SHA1())
    hm.update(msg)
    return hm.finalize()

def append_message_integrity(msg, key):
    hmac = get_message_integrity(msg, key)
    attrs = [{"message_integrity": {"hmac": hmac}, "_selector": 0x08}]
    msg = bytearray(msg)
    msg += construct(padding_0_attribute_spec, attrs)
    msg[3] = len(msg) - 20
    return msg

import binascii
def append_fingerprint(msg):
    msg = bytearray(msg)
    msg[3] = len(msg) + 8 - 20
    crc32 = binascii.crc32(msg) ^ 0x5354554e
    attrs = [{"fingerprint": {"crc32": crc32}, "_selector": 0x8028}]
    msg += construct(padding_0_attribute_spec, attrs)
    return msg
    


# v = """000100502112a4427139396c466f2b2f642f48650006000952556d333a4e6b4b36000000c0570004000003e7802a00084f02cc537c6985ff00250000002400046e001eff00080014ed4243520504f6eeefe90146a25142054ab03a4380280004f0c40a27"""
# msgm = bytes.fromhex(v)
# should_be = msgm[-28:-8].hex()
# print('should be', should_be)
# msgm = bytearray(msgm[:-8][:-24])
# key = b'7bjW8vqNNZPvdFDCa3IJ3J7E'
# print(v)
# print(append_fingerprint(append_message_integrity(msgm, key)).hex())


In [14]:
def create_response(transaction_id):
    attributes = [
        {'xor_mapped_ip': {'zeroes': 0x00,
       'family': 0x01,
       'port': target_port  ^ (magic_cookie >> 16),
       'ip': target_ip_num  ^ (magic_cookie)},
      '_selector': 0x0020}
    ]
    
    msg = construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000100000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": transaction_id,
    }
    
    msg = construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, local_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg

def create_request():
    attributes = [
        {"username": remote_ufrag+':'+local_ufrag, "_selector": 0x06},
        # {"use_candidate": "", "_selector": 0x25},
        # {"ice_controlling": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x802a},
        {"ice_controlled": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x8029},
        {"priority": {"priority": 0x6e001eff}, "_selector": 0x24},
    ]
    
    msg = construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000000000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": random.randrange(1<<96),
    }
    
    msg = construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, remote_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg


In [15]:
try:
    sock.close()
except:
    pass

sock = SocketReader(target_ip, target_port, 'udp')

sock.connect(source_port=sport)

In [ ]:
running = True
import time
from queue import Queue

dts_queue = Queue(maxsize=10)

last_received = time.time()

def handler():
    while running:
        resp = sock.read_once()
        if resp[1] == 0xfe and resp[2] == 0xfd:
            dts_queue.put(resp)
        res, rem = match(stun_header_spec, resp)
        if res['message_type'] != 0b00000000000001:
            continue
        pkg = create_response(res['transaction_id'])
        sock.send(pkg)
        print('+', end='')
        last_received = time.time()

def sender():
    while running:
        sock.send(create_request())
        print('*', end='')
        time.sleep(2 if time.time() - last_received < 6 else 0.2)

import threading
threading.Thread(target=handler).start()
threading.Thread(target=sender).start()

In [17]:
# resp = b'\x00\x01\x00T!\x12\xa4BCoSb+9vJZAMc\x00\x06\x00\x10MbsxgZXWdKc:RqEE\xc0W\x00\x04\x00\x00\x03\xe7\x80*\x00\x08\xaba\x05\x95\x95\xf0.\xd4\x00%\x00\x00\x00$\x00\x04n\x00\x1e\xff\x00\x08\x00\x14\xa2*6)\xef("\x05,~oB\x1e~,\x1dO\x0c\xc1\xea\x80(\x00\x04\xca\r\xec\x89'
# from new_parser import match
# match(stun_header_spec, resp)


In [18]:
transcript = b''

In [19]:
handshake_spec = """
8==1@_selector-24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -8@cookie()
    -16@suites(...16)
    -8@compression(...8)
    -16(
        ...@extensions(
            16==23@_selector-16@extended_master_secret()
            |16==0xff01@_selector-16@renegotiation_info(8@renegotiation_info())
            |16==10@_selector-16@ext_supported_curves(
                16@data(...16)
            )
            |16==11@_selector-16@ext_supported_points(
                8@supported_points(...8)
            )
            |16==35@_selector-16@ext_session_tickets()
            |16==13@_selector-16@ext_signature_algorithms(
                16@data(...16)
            )
            |16==14@_selector-16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            |16@selector-16@raw_extension()
        )
    )
)
|8==2@_selector-24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16(
        ...@extensions(
            16==23@_selector-16@ext_master_secret()
            |16==11@_selector-16@ext_supported_points(
                8@supported_points(...8)
            )
            |16==35@_selector-16@ext_session_tickets()
            |16==5@_selector-16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            |16==0xff01@_selector-16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            |16==14@_selector-16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            |16@selector-16@raw_extension()
        )
    )
)
|8==11@_selector-24@certificates_data(
    24@certificates(
        ...(
            24@certificate()
        )
    )
)
|8==12@_selector-24@server_key_exchange(
    8@curve_type
    -16@named_curve
    -8@pubkey()
    -16@sig_algorithm
    -16@signature()
)
|8==13@_selector-24@certificate_request(
    8@cert_types(...(8@cert_type))
    -16@sig_hash_algs(...(16@sig_hash_alg))
    -16@distinguished_names()
)
|8==14@_selector-24@server_hello_done()
|8==16@_selector-24@client_key_exchange(
    8@pubkey()
)
|8==15@_selector-24@certificate_verify(
    16@sig_algorithm
    -16@signature()
)
|8==4@_selector-24@new_session_ticket(
    4B@lifetime_hint
    -16@session_ticket()
)
|8==20@_selector-24@finished(12B@checksum)
"""

fragment_spec = """
4B@type_and_length
-16@message_sequence
-24@fragment_offset
-24@fragment()
"""

dtls_record_spec = """
8==22@_selector-16@tls_version-16@epoch-6B@sequence_number-16@tls_handshake(%s)
|8==20@_selector-16@tls_version-16@epoch-6B@sequence_number-16@change_cipher_spec_message()
"""%fragment_spec

In [20]:
client_hello = {
    'client_hello': {
        'version': 0xfefd,
        'random': bytes.fromhex('2dd93a4dd171b16cef3fd22029ab20c59938934d74b4c597b762f16ff261735b'),
        'session_id': '',
        'cookie': '',
        # 'suites': [0xc02b],
        'suites': [0xc02b, 0xc02f, 0xcca9, 0xcca8, 0xc009, 0xc013, 0xc00a, 0xc014, 0x009c, 0x002f, 0x0035],
        'compression': [0x00],
        'extensions': [
            {
                'extended_master_secret': '', '_selector': 23
            },
            {
                'renegotiation_info': {
                    'renegotiation_info': '',
                }, '_selector': 0xff01
            },
            {
                'ext_supported_curves': {
                    # 'data': [0x001d]
                    'data': [0x001d, 0x0017, 0x0018]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': [0x00]
                }, '_selector': 0x000b
            },
            {
                'ext_session_tickets': '', '_selector': 35
            },
            {
                'ext_signature_algorithms': {
                    # 'data': [0x0403]
                    'data': [0x0403, 0x0804, 0x0401, 0x0503, 0x0805, 0x0501, 0x0806, 0x0601, 0x0201]
                }, '_selector': 0x000d
            },
            {
                'use_srtp': {
                    # 'data': [0x0001],
                    'data': [0x0001, 0x0008, 0x0007],
                    'mki': ''
                }, '_selector': 14
            }
        ]
    },
    '_selector': 0x01
}

client_hello_encoded = construct(handshake_spec, client_hello)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_hello_encoded[:4],
        'message_sequence': 0,
        'fragment_offset': 0,
        'fragment': client_hello_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 0,
    "_selector": 22
}
dtls_record_encoded = construct(dtls_record_spec, dtls_record)

dtls_record_encoded

b'\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x90\x01\x00\x00\x84\x00\x00\x00\x00\x00\x00\x00\x84\xfe\xfd-\xd9:M\xd1q\xb1l\xef?\xd2 )\xab \xc5\x998\x93Mt\xb4\xc5\x97\xb7b\xf1o\xf2as[\x00\x00\x00\x16\xc0+\xc0/\xcc\xa9\xcc\xa8\xc0\t\xc0\x13\xc0\n\xc0\x14\x00\x9c\x00/\x005\x01\x00\x00D\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\x08\x00\x06\x00\x1d\x00\x17\x00\x18\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\r\x00\x14\x00\x12\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x02\x01\x00\x0e\x00\t\x00\x06\x00\x01\x00\x08\x00\x07\x00'

In [21]:
transcript += dtls_record_encoded[13:]
sock.send(dtls_record_encoded)

In [22]:
# msg = bytes.fromhex("16fefd00000000000000000050020000440000000000000044fefd666d5aac48243def6a5466b05ba62cbf15f4eea136c99df59f782732ef9d558900c02b00001c00170000ff01000100000b0002010000230000000e0005000200010016fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda0030201020209008bff228a3f27c618300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303932315a170d3234303731353039303932315a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004a7990ef7c6e5f1a8c29e5cdb4b3e4cb1dc376b48ffd392751dc6fd763de583ee77a65da77341fbeed4e83fb1a09d4f1c621c89d9d974e50a0339ed4d380c46bb300a06082a8648ce3d0403020348003045022100a07ff3282ccb96a42387a8851d0b26a42a5f1082789fe8f682df50fbccb35a8202201dc64921ff9358f5bdec2a644c0597312f5cea62f295b875daf0e6bb1fe3c3e916fefd0000000000000002007c0c000070000200000000007003001d20eb7088c2a60f227ef9bdb01a5082a34048ff182b9e142afcb40c07b368fea016040300483046022100fec9df11479090a9553f1497c7707e4146100af1f9e5f36c60f9a4c421c5f419022100e494e6bb2765926893edd08f028cf692d69e7534affa89756802768a7bdc956816fefd000000000000000300250d00001900030000000000190201400012040308040401050308050501080606010201000016fefd0000000000000004000c0e0000000004000000000000")
msg = dts_queue.get()

In [23]:
packet, rem = match(dtls_record_spec, msg)
transcript += msg[13:-len(rem)]
packet

{'_selector': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x000000000000,
 'tls_handshake': {'type_and_length': 0x02000044,
  'message_sequence': 0x0000,
  'fragment_offset': 0x000000,
  'fragment': 0xfefd668be17ff346db688bb2d247e88965d6d4ce89b32fc043cfaee6511d92847df400c02b00001c00170000ff01000100000b0002010000230000000e00050002000100}}

In [24]:
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
server_hello, _ = match(handshake_spec, record)
server_hello

{'_selector': 2,
 'server_hello': {'version': 0xfefd,
  'random': 0x668be17ff346db688bb2d247e88965d6d4ce89b32fc043cfaee6511d92847df4,
  'session_id': 0x,
  'suite': 0xc02b,
  'compression': 0,
  'extensions': [{'_selector': 0x0017, 'ext_master_secret': 0x},
   {'_selector': 0xff01,
    'ext_renegotiation_info': {'secure_renegotitation': 0x}},
   {'selector': 0x000b, 'raw_extension': 0x0100},
   {'_selector': 0x0023, 'ext_session_tickets': 0x},
   {'selector': 0x000e, 'raw_extension': 0x0002000100}]}}

In [25]:
# rem = bytes.fromhex('16fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda003020102020900b388ba8f0397e380300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303931365a170d3234303731353039303931365a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d030107034200045d4f0a2e4e3e0bfe83012c62d4649a43fa9bb46aa4579fa59b739e6463a0a8c0395545195f81f8895480128cda85a9c79699baca8b413065c7ef013c226be5e6300a06082a8648ce3d040302034800304502206e8136052a4d8689988d3f1205882d1d224cd9b2543c1e9463ac9bae5aee87f90221009c4d8b5e119541c461c4d704121abb097ee983dddb0c7ea4c8fd06ba08623cea')
packet, rem1 = match(dtls_record_spec, rem)
transcript += rem[13:-len(rem1)]
packet

{'_selector': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x000000000001,
 'tls_handshake': {'type_and_length': 0x0b00011f,
  'message_sequence': 0x0001,
  'fragment_offset': 0x000000,
  'fragment': 0x00011c000119308201153081bda003020102020900e9f5564ad0e4d673300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303730373132353430385a170d3234303830373132353430385a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d030107034200045d549c50f60bc90f648909c11950a29bfb1df3667844d4819914b558f291f259d4730dc90e8b8147a0c5c0c78e5c6cdc4624b825b7dfd61197a05c348407ad79300a06082a8648ce3d040302034700304402202d53605401a678e90e204758a5c823b683e6b571de3bb84bf4a39730716608b602207ac82322b53c8c4ab54a5644e850e96688830916d4bc193531cc1158622be470}}

In [26]:
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
peer_cert, _ = match(handshake_spec, record)
peer_cert

{'_selector': 11,
 'certificates_data': {'certificates': {'certificate': [0x308201153081bda003020102020900e9f5564ad0e4d673300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303730373132353430385a170d3234303830373132353430385a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d030107034200045d549c50f60bc90f648909c11950a29bfb1df3667844d4819914b558f291f259d4730dc90e8b8147a0c5c0c78e5c6cdc4624b825b7dfd61197a05c348407ad79300a06082a8648ce3d040302034700304402202d53605401a678e90e204758a5c823b683e6b571de3bb84bf4a39730716608b602207ac82322b53c8c4ab54a5644e850e96688830916d4bc193531cc1158622be470]}}}

In [27]:

first_server_cert = None
for cert in peer_cert['certificates_data']['certificates']['certificate']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert._value('bytes'))
    if first_server_cert is None:
        first_server_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            e9:f5:56:4a:d0:e4:d6:73
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jul  7 12:54:08 2024 GMT
            Not After : Aug  7 12:54:08 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:5d:54:9c:50:f6:0b:c9:0f:64:89:09:c1:19:50:
                    a2:9b:fb:1d:f3:66:78:44:d4:81:99:14:b5:58:f2:
                    91:f2:59:d4:73:0d:c9:0e:8b:81:47:a0:c5:c0:c7:
                    8e:5c:6c:dc:46:24:b8:25:b7:df:d6:11:97:a0:5c:
                    34:84:07:ad:79
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:44:02:20:2d:53:60:54:01:a6:78:e9:0e:20:47:58:a5:c8:
        23:b6:83:e6:b5:71:de:3b:b8:4b:f4

In [28]:
first_server_cert.digest('sha256')
# TODO: assert digest

b'FB:53:9E:49:62:01:25:B8:4F:3E:2B:F7:8C:C0:3D:FC:52:35:6E:9A:6A:4D:3B:26:F8:71:1C:FC:6E:4D:F0:F4'

In [29]:
packet, rem2 = match(dtls_record_spec, rem1)
transcript += rem1[13:-len(rem2)]
packet

{'_selector': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x000000000002,
 'tls_handshake': {'type_and_length': 0x0c000070,
  'message_sequence': 0x0002,
  'fragment_offset': 0x000000,
  'fragment': 0x03001d20af5377ca4f6de05a0f76d3d76f8b8fe59dba452129a33734946da4277c047b23040300483046022100ed608c23a6739c46da09b70e2c3c891f45a76dc9da2f9ba4a7e227e2137b8cbf02210094315400add760239b3712cef7e21a466bc793f0e9c41ca5e3a4bf069d2b32b0}}

In [30]:
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
server_key, _ = match(handshake_spec, record)
server_key

{'_selector': 12,
 'server_key_exchange': {'curve_type': 3,
  'named_curve': 0x001d,
  'pubkey': 0xaf5377ca4f6de05a0f76d3d76f8b8fe59dba452129a33734946da4277c047b23,
  'sig_algorithm': 0x0403,
  'signature': 0x3046022100ed608c23a6739c46da09b70e2c3c891f45a76dc9da2f9ba4a7e227e2137b8cbf02210094315400add760239b3712cef7e21a466bc793f0e9c41ca5e3a4bf069d2b32b0}}

In [31]:
server_key_sign_spec = """32B@client_random-32B@server_random-8@curve_type-16@named_curve-8@pubkey()"""
server_key_sign = {
    "client_random": client_hello['client_hello']['random'],
    "server_random": server_hello['server_hello']['random'],
    "curve_type": server_key['server_key_exchange']['curve_type'],
    "named_curve": server_key['server_key_exchange']['named_curve'],
    "pubkey": server_key['server_key_exchange']['pubkey'],
}

enc = construct(server_key_sign_spec, server_key_sign)

first_server_cert.get_pubkey().to_cryptography_key().verify(server_key['server_key_exchange']['signature']._value('bytes'), enc, ec.ECDSA(hashes.SHA256()))


In [32]:
packet, rem3 = match(dtls_record_spec, rem2)
transcript += rem2[13:-len(rem3)]
packet

{'_selector': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x000000000003,
 'tls_handshake': {'type_and_length': 0x0d000019,
  'message_sequence': 0x0003,
  'fragment_offset': 0x000000,
  'fragment': 0x02014000120403080404010503080505010806060102010000}}

In [33]:
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
certificate_request, _ = match(handshake_spec, record)
certificate_request

{'_selector': 13,
 'certificate_request': {'cert_types': {'cert_type': [1, 64]},
  'sig_hash_algs': {'sig_hash_alg': [0x0403,
    0x0804,
    0x0401,
    0x0503,
    0x0805,
    0x0501,
    0x0806,
    0x0601,
    0x0201]},
  'distinguished_names': 0x}}

In [34]:
packet, rem4 = match(dtls_record_spec, rem3)
transcript += rem3[13:]
packet

{'_selector': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x000000000004,
 'tls_handshake': {'type_and_length': 0x0e000000,
  'message_sequence': 0x0004,
  'fragment_offset': 0x000000,
  'fragment': 0x}}

In [35]:
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
server_hello_done, _ = match(handshake_spec, record)
server_hello_done

{'_selector': 14, 'server_hello_done': 0x}

In [36]:
dtls_message_encoded = b''

In [37]:
certificate = {
    'certificates_data':
        {
            'certificates':
                 [{'certificate': OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_ASN1, local_cert)}]
        },
 '_selector': 11
}

certificate_encoded = construct(handshake_spec, certificate)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_encoded[:4],
        'message_sequence': 1,
        'fragment_offset': 0,
        'fragment': certificate_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 1,
    "_selector": 22
}
dtls_record_encoded = construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [38]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey

client_private_key = X25519PrivateKey.generate()
client_public_key = client_private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)


In [39]:

client_key_exchange = {
    'client_key_exchange': {
        'pubkey': client_public_key,
    },
 '_selector': 16
}

client_key_exchange_encoded = construct(handshake_spec, client_key_exchange)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_key_exchange_encoded[:4],
        'message_sequence': 2,
        'fragment_offset': 0,
        'fragment': client_key_exchange_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 2,
    "_selector": 22
}
dtls_record_encoded = construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [40]:
signature = own_cert_private_key.sign(transcript, ec.ECDSA(hashes.SHA256()))

In [41]:

certificate_verify = {
    'certificate_verify': {
        'sig_algorithm': 0x0403,
        'signature': signature,
    },
 '_selector': 15
}

certificate_verify_encoded = construct(handshake_spec, certificate_verify)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_verify_encoded[:4],
        'message_sequence': 3,
        'fragment_offset': 0,
        'fragment': certificate_verify_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 3,
    "_selector": 22
}
dtls_record_encoded = construct(dtls_record_spec, dtls_record)
# transcript += dtls_record_encoded[13:]
certificate_verify_message_encoded = dtls_record_encoded
dtls_message_encoded += dtls_record_encoded

In [42]:

dtls_record = {
    "tls_version": 0xfefd,
    "epoch": 0,
    "sequence_number": 4,
    "change_cipher_spec_message": b'\x01',
    "_selector": 20
}
dtls_record_encoded = construct(dtls_record_spec, dtls_record)
dtls_message_encoded += dtls_record_encoded

In [43]:
from cryptography.hazmat.primitives import hashes, hmac

def HMAC_SHA256(secret, val):
    h = hmac.HMAC(secret, hashes.SHA256())
    h.update(val)
    return h.finalize()

def P_SHA256(secret, seed, l):
    b = bytearray()
    A = HMAC_SHA256(secret, seed)
    while len(b) < l:
        b += HMAC_SHA256(secret, A + seed)
        A = HMAC_SHA256(secret, A)
    return b[:l]

def PRF(secret, label, seed, l):
    return P_SHA256(secret, label + seed, l)

In [44]:
def calculate_sha256(data):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(data)
    return digest.finalize()

In [45]:
shared_key = client_private_key.exchange(X25519PublicKey.from_public_bytes(server_key['server_key_exchange']['pubkey']._value('bytes')))
print("calculated shared key", shared_key)

calculated shared key b'\x1e\x01\xbf\r\x9dHT\xe7\xd7\xc3\xa6Ga\xa7\x928+\xa2\x9a\xe9Y\xf6$\xb1\x9d\xc7@\xfbo\x1e\xf4c'


In [46]:
master_key = PRF(
    shared_key,
    b"extended master secret",
    # client_hello['client_hello']['random'].bytes() + server_hello['server_hello']['random'],
    calculate_sha256(transcript),
    48,
)
master_key.hex()

'54885a5ebd61d76a58866d3520b12a8a9c62dcc0ddcddc2853851b47107d801a6b48065fad713a586c87ed0044bb6d6f'

In [95]:
keyLen = 16
ivLen  = 4

raw_keys = PRF(
    master_key,
    b"key expansion",
    server_hello['server_hello']['random'] + client_hello['client_hello']['random'],
    40,
)

client_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

server_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

client_write_iv = raw_keys[:ivLen]
raw_keys = raw_keys[ivLen:]

server_write_iv = raw_keys[:ivLen]

In [48]:
transcript += certificate_verify_message_encoded[13:]

In [49]:
client_verify_data = PRF(master_key, b"client finished", calculate_sha256(transcript), 12)
client_verify_data.hex()

'cc28c28173d1a2326cc23112'

In [50]:
encrypted_dtls_record_spec = """
8@content_type-16@tls_version-16@epoch-6B@sequence_number-16@encrypted_data()
"""

In [51]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

aad_spec = """16@epoch-6B@sequence_number-8@content_type-16@tls_version-16@data_length"""

def decrypt_packet(packet, key, iv):
    encrypted = packet['encrypted_data']

    aad = packet.copy()
    aad['data_length'] = len(encrypted) - 16 - 8
    aad_encoded = bytearray(construct(aad_spec, aad))
    aad_encoded[:8] = encrypted[:8]

    cipher = AESGCM(key)
    all_server_cert_packet_encoded = cipher.decrypt(iv + encrypted[:8], encrypted[8:], aad_encoded)
    if packet['content_type'] == 0x16:
        all_server_cert_packet, _ = match(fragment_spec, all_server_cert_packet_encoded)
        packet['tls_handshake'] = all_server_cert_packet
    else:
        packet['decrypted'] = all_server_cert_packet_encoded
    return packet
    

def encrypt_packet(packet, key, iv):
    if packet['content_type'] == 0x16:
        to_encrypt = construct(fragment_spec, packet['tls_handshake'])
    else:
        to_encrypt = packet['decrypted']

    aad = packet.copy()
    aad['data_length'] = len(to_encrypt)
    # aad['sequence_number'] = aad['tls_handshake']['message_sequence']
    aad_encoded = construct(aad_spec, aad)
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(iv + aad_encoded[:8], to_encrypt, aad_encoded)
    packet['encrypted_data'] = aad_encoded[:8] + encrypted_app_data
    packet['data_length'] = len(packet['encrypted_data'])
    return packet

In [52]:
finshed_record = {
    "tls_version": 0xfefd,
    "epoch": 1,
    "sequence_number": 0,
    "tls_handshake": {
        'type_and_length': 0x1400000c,
      'message_sequence': 4,
      'fragment_offset': 0x000000,
      'fragment': bytes(client_verify_data)
    },
    "content_type": 22,
    "_selector": 22,
}
encrypted_finished_record = encrypt_packet(finshed_record, client_write_key, client_write_iv)
print(encrypted_finished_record)
encrypted_dtls_record_encoded = construct(encrypted_dtls_record_spec, encrypted_finished_record)

# transcript += dtls_record_encoded[13:]
dtls_message_encoded += encrypted_dtls_record_encoded

{'tls_version': 65277, 'epoch': 1, 'sequence_number': 0, 'tls_handshake': {'type_and_length': 335544332, 'message_sequence': 4, 'fragment_offset': 0, 'fragment': b'\xcc(\xc2\x81s\xd1\xa22l\xc21\x12'}, 'content_type': 22, '_selector': 22, 'encrypted_data': b'\x00\x01\x00\x00\x00\x00\x00\x00\x8b\x83\xa4\t\x95\x7f\xf2}\x80\x86\xd5\xfb\x00\xc8G\x85\xac}\x12t\x03$A\xde?I\xf0\xd0\xbc\x01\x0c\xb5\xb1k\x90\xe4M\xf3\xcc\xaf', 'data_length': 48}


In [53]:
aaa = construct(dtls_record_spec, finshed_record)
print(aaa[13:])
transcript += aaa[13:]
# transcript = transcript[:-len(aaa)+13]

b'\x14\x00\x00\x0c\x00\x04\x00\x00\x00\x00\x00\x0c\xcc(\xc2\x81s\xd1\xa22l\xc21\x12'


In [54]:
sock.send(dtls_message_encoded)
dtls_message_encoded = b''

In [58]:
msg = dts_queue.get()
msg

b"\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00\n\x01\xe2\x04\x00\x01\xd6\x00\x05\x00\x00\x00\x00\x01\xd6\x00\x00\x1c\x1f\x01\xd0\xe7\xbd\x99\xe5\x1du\xa1\x12\xe3\x93\x04A\xe3\x00*n'mW/j\xbb\xf5\xe0I*\x8d\xd8\xbaK\x91\x8e|\xd0\xe0e\xf6\xbc\xb0P\xa9\x04\xfef\x14=\xbb\x8c0\xe3\x9cM\xe4\xfa\xdaJ\x15\r\x11\x015\xa0\x0e\x03<\x00\n}\x16\xb0\x8cTu\xb9ip\x0fR\xc1\xc6\x1c[\xd1\xa93^\xcc&\xa1\xb1\x87\x0fE\x84\xc1`\xc1Q!-$N\xfe(\xfb\xc1\x8e/\xe9Lp\xc0b\xde\t\x88\xe3\xad\xd4m\xe4\xbc\\\xec\x9e\x07{\xf2b\xd7T\x0b\x01@\x99\xb3\xfbg1i\xab#7\xfd\xe8\xca\x7fB\xbe\xddf6\xd1{R\xdb!\x18V\x91ZU*\xd5\x97\xe4\xcc\xd5\x9d\xcf\xf29V\x1d:\x98K\xdc\xd9[l\xcd\xfe3\xfe\xedAv)\x99\xcf[\xcf\xed\xf3fm@k\xafU\x01S\x1b\x97\x08\xd7\xdc\x9dn\x1dw\x11*\xf2\xac\xa6\x07G\xb8\xa9\x89K\xceSz\xc5f\x1eu\x02\xee\xf9Q\xbcFl8\xa8\xad:\xe6\xaeO\xf7\x19\xa9\xb4\x83\x98B6!i\x1f\xcc\xae/\xf8<1\xdeR\xf59\x7f\x19+sY\x02\xadb}USV\xd4,.\x9d\xc7\xa2\xf5\xb4\xc5\xc4\xbc\xe0\x93\x00\x97Z''\xbf\xcc1\xe7\x90\xe4\xb5\xbd\x9f\xdc\xc6\xad\xf1\x9c)l\xf

In [59]:
packet, rem = match(dtls_record_spec, msg)
record = packet['tls_handshake']['type_and_length'] + packet['tls_handshake']['fragment']
peer_cert, _ = match(handshake_spec, record)
peer_cert

{'_selector': 4,
 'new_session_ticket': {'lifetime_hint': 0x00001c1f,
  'session_ticket': 0xe7bd99e51d75a112e3930441e3002a6e276d572f6abbf5e0492a8dd8ba4b918e7cd0e065f6bcb050a904fe66143dbb8c30e39c4de4fada4a150d110135a00e033c000a7d16b08c5475b969700f52c1c61c5bd1a9335ecc26a1b1870f4584c160c151212d244efe28fbc18e2fe94c70c062de0988e3add46de4bc5cec9e077bf262d7540b014099b3fb673169ab2337fde8ca7f42bedd6636d17b52db211856915a552ad597e4ccd59dcff239561d3a984bdcd95b6ccdfe33feed41762999cf5bcfedf3666d406baf5501531b9708d7dc9d6e1d77112af2aca60747b8a9894bce537ac5661e7502eef951bc466c38a8ad3ae6ae4ff719a9b48398423621691fccae2ff83c31de52f5397f192b735902ad627d555356d42c2e9dc7a2f5b4c5c4bce09300975a2727bfcc31e790e4b5bd9fdcc6adf19c296cf7edd4bc7ad2766bd07e4bd59bf54f837bb4c624cfc51256e8d67e5136668b51694f95435e82f3185cc118d2e52b18108df316d76ea9b0f71aba859c5e98a513a8496901514f7947fa0deb37ad0042cc8ee3211c3540c6c3fe97669a272db9edd2776c7f65cfd592798e5f79410d78e0b3dc915c5d9c456e459c63bc0be48e1d0ff5cced3693da211778d653372129

In [60]:
if 'new_session_ticket' not in peer_cert:
    raise RuntimeError("repeated package, retry from dts_queue.get()")
transcript += msg[13:-len(rem)]


In [61]:
packet, rem1 = match(dtls_record_spec, rem)
packet

{'_selector': 20,
 'tls_version': 0xfefd,
 'epoch': 0x0000,
 'sequence_number': 0x00000000000b,
 'change_cipher_spec_message': 1}

In [62]:
packet, rem2 = match(encrypted_dtls_record_spec, rem1)
packet

{'content_type': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x00010000000000005b0ab57972d1029fd25d0706bb1cdffb3eacb3d8ce740c9dd00b3b643fbce5e394a96e5423b9187c}

In [63]:
finished_fragment = decrypt_packet(
    packet,
    server_write_key,
    server_write_iv,
)
finished_fragment

{'content_type': 22,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x00010000000000005b0ab57972d1029fd25d0706bb1cdffb3eacb3d8ce740c9dd00b3b643fbce5e394a96e5423b9187c,
 'tls_handshake': {'type_and_length': 0x1400000c,
  'message_sequence': 0x0006,
  'fragment_offset': 0x000000,
  'fragment': 0xa7481124c0158007fc45c477}}

In [64]:
server_verify_data = PRF(master_key, b"server finished", calculate_sha256(transcript), 12)
server_verify_data.hex()

'a7481124c0158007fc45c477'

In [65]:
assert finished_fragment['tls_handshake']['fragment'] == server_verify_data, "server verify data mismatch"

In [66]:
import crc

In [67]:
sctp_chunks_spec = """
8==1@_selector-8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
|8==2@_selector-8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
|8==10@_selector-8@chunk_flags
    -16@chunk_length
    -@cookie
|8==11@_selector-8@chunk_flags
    -16@chunk_length
|8==0@_selector-4@res
    -1@i-1@u-1@b-1@e
    -16@chunk_length
    -32@tsn
    -16@s-16@n
    -32@payload_protocol_id
    -@data
|8==3@_selector-8@flags
    -16@chunk_length
    -32@tsn_ack
    -32@a_rwnd
    -16@n_gap_ack_blocks
    -16@n_duplicate_tsns
    -@gap_ack_duplicate_tsns
|8==4@_selector-8@flags
    -16@chunk_length
    -@optional_attributes_encoded
|8==5@_selector-8@flags
    -16@chunk_length
    -@optional_attributes_encoded
"""

optional_header_spec = """16@type-16@length"""
sctp_chunk_header_spec = "8@chunk_type-8@chunk_flags-16@chunk_size"

def parse_sctp_chunk(sctp_chunk):
    sctp_chunk_header, _ = match(sctp_chunk_header_spec, sctp_chunk)
    data_len = sctp_chunk_header['chunk_size']._value('int')
    sctp_chunk_value, sctp_chunk = sctp_chunk[:data_len], sctp_chunk[math.ceil(data_len /4) * 4:]
    res, _ = match(sctp_chunks_spec, sctp_chunk_value)
    if 'optional_attributes_encoded' in res:
        oares = res['optional_attributes_encoded']
        res['optional_attributes'] = []
        while oares:
            optional_attribute_header, _ = match(optional_header_spec, oares)
            optional_attribute_header['data'] = oares[4:optional_attribute_header['length']._value('int')]
            oares = oares[optional_attribute_header['length']._value('int'):]
            res['optional_attributes'].append(optional_attribute_header)
    return res, sctp_chunk
            
        

In [68]:
sctp_common_header_spec = """
16@source_port
-16@dst_port
-32@verification_tag
-32@checksum"""

def get_sctp_message(dtls_message):
    packet, _ = match(encrypted_dtls_record_spec, dtls_message)
    decrypted = decrypt_packet(
        packet,
        server_write_key,
        server_write_iv,
    )
    raw = decrypted['decrypted']
    raw_clone = bytearray(raw)
    raw_clone[8:12] = b'\0\0\0\0'
    crc_res = construct('32@data', {'data': crc.crc(raw_clone)})[::-1]
    
    sctp_header, rem = match(sctp_common_header_spec, raw)
    assert crc_res == sctp_header['checksum'], "invalid checksum"
    chunks = []
    while rem:
        chunk, rem = parse_sctp_chunk(rem)
        chunks.append(chunk)
    return {"header": sctp_header, "chunks": chunks, "epoch": decrypted["epoch"], "sequence_number": decrypted["sequence_number"]}

In [69]:
optional_full_header_spec = """16@type-16@length-@data"""
def construct_sctp_chunk(sctp_chunk):
    if 'optional_attributes' in sctp_chunk:
        sctp_chunk['optional_attributes_encoded'] = b''
        for attr in sctp_chunk['optional_attributes']:
            attr['length'] = len(attr['data']) + 4
            sctp_chunk['optional_attributes_encoded'] += construct(optional_full_header_spec, attr)
    sctp_chunk['chunk_length'] = 0
    res = construct(sctp_chunks_spec, sctp_chunk)
    sctp_chunk['chunk_length'] = len(res)
    res = construct(sctp_chunks_spec, sctp_chunk)
    if sctp_chunk['chunk_length'] % 4 != 0:
        res += b'\0' * (4-(sctp_chunk['chunk_length']%4))
    return res
    

In [70]:
def create_sctp_message(sctp_message):
    sctp_message['header']['checksum'] = 0
    buff = construct(sctp_common_header_spec, sctp_message['header'])
    for chunk in sctp_message['chunks']:
        buff += construct_sctp_chunk(chunk)

    buff = bytearray(buff)
    crc_res = construct('32@data', {'data': crc.crc(buff)})[::-1]
    buff[8:12] = crc_res
    
    packet = {
        "tls_version": 0xfefd,
        "epoch": sctp_message["epoch"],
        "sequence_number": sctp_message["sequence_number"],
        "decrypted": bytes(buff),
        "content_type": 23,
        "_selector": 23,
    }
    encrypted = encrypt_packet(
        packet,
        client_write_key,
        client_write_iv,
    )
    return construct(encrypted_dtls_record_spec, encrypted)
    

In [71]:
while True:
    msg = dts_queue.get()
    if msg[0] != 0x17:
        print('not an app data, retrying')
        continue
    print('got app data')
    break

sctp_init = get_sctp_message(msg)
client_verification_tag = sctp_init['chunks'][0]['initiate_tag']
sctp_init

got app data


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0x00000000,
  'checksum': 0x490f13e8},
 'chunks': [{'_selector': 1,
   'chunk_flags': 0,
   'chunk_length': 0x001e,
   'initiate_tag': 0xc9066be1,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': 0xee7dd972,
   'optional_attributes_encoded': 0xc00000048008000682c0,
   'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
    {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000001}

In [72]:
tsn = 0xe306eef2
own_dtls_seq = 2

In [73]:
# sctp_init = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': 0x00000000,
#   'checksum': 0x9e2630c2},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 0x01,
#    'chunk_length': 0x001e,
#    'initiate_tag': 0x6ee08c30,
#    'a_rwnc': 0x00500000,
#    'num_out_streams': 0xffff,
#    'num_in_streams': 0xffff,
#    'initial_tsn': tsn,
#    'optional_attributes_encoded': 0xc00000048008000682c0,
#    'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
#     {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000007})
# sock.send(sctp_init)

In [74]:
sctp_init_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x378def6c},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 0x02,
   'chunk_length': 0x0052,
   'initiate_tag': 0xff448058,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': tsn,
   'optional_attributes_encoded': 0x0007003164635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00000000c00000048008000682c0,
   'optional_attributes': [{'type': 0x0007,
     'length': 0x0031,
     'data': b'dcSCTP00n\xe0\x8c0\xffD\x80X\x91p\x92\x7f\xe3\x06\xee\xf2\x00P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\xff\xff\xff\xff\x00'},
    {'type': 0x0000,
     'length': 0x00c0,
     'data': b'\x00\x00\x04\x80\x08\x00\x06\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_init_ack)


In [75]:
# while True:
#     msg = dts_queue.get()
#     pkt = get_sctp_message(msg)
#     if pkt['chunks'][0]['_selector'].int() != 2:
#         continue
#     print('found init ack')    
#     cookie = next(i['data'] for i in pkt['chunks'][0]['optional_attributes'] if i['type'].int() == 7)
#     server_verification_tag = pkt['header']['verification_tag']
#     break
# pkt

In [76]:
while True:
    msg = dts_queue.get()
    pkt = get_sctp_message(msg)
    if pkt['chunks'][0]['_selector'] != 10:
        print('not found cookie echo, got', pkt['chunks'][0]['_selector'], "instead")    
        continue
    print('found cookie echo')
    break
pkt

not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
found cookie echo


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xa804049c},
 'chunks': [{'_selector': 10,
   'chunk_flags': 0,
   'chunk_length': 0x0031,
   'cookie': 0x64635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000005}

In [77]:
# sctp_cookie_echo = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': server_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 10,
#     'cookie': cookie}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_echo)

In [78]:
sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 11}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_cookie_ack)

In [79]:
# sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': client_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 11}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_ack)

In [80]:
sctp_data_sequence_number = 0

In [81]:
data_send = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x14fad29b},
 'chunks': [{'res': 0b0000,
   '_selector': 0x00,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': tsn,
   's': 0x0000,
   'n': sctp_data_sequence_number,
   'payload_protocol_id': 0x00000033,
   'data': "hello, my name is mie2"}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
tsn += 1
sctp_data_sequence_number += 1
sock.send(data_send)

In [82]:
msg = dts_queue.get()
data_ack_received = get_sctp_message(msg)
data_ack_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xe02faf3e},
 'chunks': [{'_selector': 3,
   'flags': 0,
   'chunk_length': 0x0010,
   'tsn_ack': 0xe306eef2,
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': 0x}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000006}

In [83]:
print('waiting for a message, please send a message from the other side...')
msg = dts_queue.get()
data_received = get_sctp_message(msg)
data_received

waiting for a message, please send a message from the other side...


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0x0cb06fc9},
 'chunks': [{'_selector': 0,
   'res': 0b0000,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': 0xee7dd972,
   's': 0x0000,
   'n': 0x0000,
   'payload_protocol_id': 0x00000033,
   'data': 0x617364}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000007}

In [84]:
sack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0xe02faf3e},
 'chunks': [{'flags': 0x00,
   '_selector': 0x03,
   'chunk_length': 0x0010,
   'tsn_ack': data_received['chunks'][-1]['tsn'],
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': ''}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sack)

In [85]:
print('waiting for a heartbeat, you can just wait...')
msg = dts_queue.get()
heartbeat_received = get_sctp_message(msg)
heartbeat_received

waiting for a heartbeat, you can just wait...


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xbd662d46},
 'chunks': [{'_selector': 4,
   'flags': 0,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c000000000653f318,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': b'\x00\x00\x00\x00\x06S\xf3\x18'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000008}

In [86]:
heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x30743be0},
 'chunks': [{'flags': 0x00,
   '_selector': 0x05,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c00000000a92c0797,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': heartbeat_received['chunks'][0]['optional_attributes'][0]['data']}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(heartbeat_ack)

In [87]:
background_sctp_handler_running = False
try:
    background_sctp_thread.join()
except:
    pass
def message_handler(msg):
    global own_dtls_seq
    decoded = get_sctp_message(msg)
    ret = []
    for chunk in decoded['chunks']:
        if chunk['_selector'] == 0: # data
            sack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0xe02faf3e},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x03,
                    'chunk_length': 0x0010,
                    'tsn_ack': chunk['tsn'],
                    'a_rwnd': 0x00480000,
                    'n_gap_ack_blocks': 0x0000,
                    'n_duplicate_tsns': 0x0000,
                    'gap_ack_duplicate_tsns': ''}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(sack)
            ret.append(chunk['data'])
        elif chunk['_selector'] == 4: # heartbeat
            heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0x30743be0},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x05,
                    'chunk_length': 0x0010,
                    'optional_attributes_encoded': 0x0001000c00000000a92c0797,
                    'optional_attributes': [{'type': 0x0001,
                    'length': 0x000c,
                    'data': chunk['optional_attributes'][0]['data']}]}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(heartbeat_ack)
    return ret


In [88]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        print(msg)
        print(message_handler(msg))
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()

b'\x17\xfe\xfd\x00\x01\x00\x00\x00\x00\x00\t\x004\x00\x01\x00\x00\x00\x00\x00\t\xe1\x98(\xad\xc0@\x8dy\xc0\xd8\xdd\xc4\xb0w"w\x85\xf2`\xf6\x10\xdf\n5E\xe7y\xcb\x88\x8dS\x07u\x94\xe1\xae\x82\xdb\xb3Rs\xbd\xa5\xf2'
[]
exiting


In [89]:
def send_message(data):
    global own_dtls_seq
    global tsn
    global sctp_data_sequence_number
    data_send = create_sctp_message({'header': {'source_port': 0x1388,
        'dst_port': 0x1388,
        'verification_tag': client_verification_tag,
        'checksum': 0x14fad29b},
        'chunks': [{'res': 0b0000,
            '_selector': 0x00,
            'i': 0b0,
            'u': 0b0,
            'b': 0b1,
            'e': 0b1,
            'chunk_length': 0x0013,
            'tsn': tsn,
            's': 0x0000,
            'n': sctp_data_sequence_number,
            'payload_protocol_id': 0x00000033,
            'data': data}],
        'epoch': 0x0001,
        'sequence_number': own_dtls_seq})
    own_dtls_seq += 1
    tsn += 1
    sctp_data_sequence_number += 1
    sock.send(data_send)

In [90]:
send_message("hello a second time")

In [91]:
background_sctp_handler_running = False
background_sctp_thread.join()

In [92]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        ret = message_handler(msg)
        for i in ret:
            print('>', i._value('str'))
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()
while True:
    send_message(input())

 asd


> sfd


KeyboardInterrupt: Interrupted by user